1.Install all required libraries

In [ ]:
!pip install langchain-google-genai langchain beautifulsoup4 requests \
sentence-transformers chromadb pypdf python-docx unstructured

In [ ]:
!pip install --upgrade langchain langchain-community langchain-google-genai \
    google-generativeai beautifulsoup4 requests sentence-transformers chromadb \
    pypdf python-docx unstructured

  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pro

2. Setup Imports and Gemini API

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader

# Set Gemini API key
os.environ["GOOGLE_API_KEY"] = "your-API-key-here"


3. Fetch and Parse Web Content

In [ ]:
def fetch_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    for tag in soup(['script', 'style']):
        tag.decompose()
    return soup.get_text(separator='\n', strip=True)

# List of college URLs
urls = [
    "https://www.thapar.edu/",
    "https://www.thapar.edu/admissions/pages/admissions",
    "https://www.thapar.edu/academics",
    "https://portal.thapar.edu/aboutus/pages/university",
    "https://www.thapar.edu/students/pages/fee-circular-and-fee-chart",
    "https://www.thapar.edu/admissions/pages/scholarships",
    "https://www.thapar.edu/students/pages/hostels",
    "https://www.thapar.edu/faculties",
    "https://www.thapar.edu/aboutus/pages/directory",
    "https://www.thapar.edu/students/pages/introduction-sports",
    "https://www.thapar.edu/placements"
]

web_texts = [fetch_text_from_url(url) for url in urls]
web_docs = [Document(page_content=text, metadata={"source": urls[i]}) for i, text in enumerate(web_texts)]


4. Load and Parse Document Files

In [ ]:
import io
import os
import tempfile
from google.colab import files
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader

# Lists to hold all loaded documents
all_docs = []

# Upload via Colab interface
uploaded = files.upload()

# Helper function to load content using the correct loader
def load_file_with_loader(file_bytes, filename):
    suffix = '.' + filename.split('.')[-1]

    # Save uploaded file temporarily
    with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp:
        tmp.write(file_bytes.read())
        tmp_path = tmp.name

    # Choose the correct loader
    if filename.endswith('.pdf'):
        loader = PyPDFLoader(tmp_path)
    elif filename.endswith('.docx'):
        loader = Docx2txtLoader(tmp_path)
    elif filename.endswith('.txt'):
        loader = TextLoader(tmp_path)
    else:
        raise ValueError(f"❌ Unsupported file type: {filename}")

    return loader.load()

# Process each uploaded file
for filename in uploaded:
    print(f"📄 Loading: {filename}")
    file_bytes = io.BytesIO(uploaded[filename])
    docs = load_file_with_loader(file_bytes, filename)
    all_docs.extend(docs)

print(f"\n✅ Successfully loaded {len(all_docs)} document chunks from {len(uploaded)} file(s).")


Saving 01 ABBREVIATIONS USED.pdf to 01 ABBREVIATIONS USED.pdf
Saving 02 PREFACE.pdf to 02 PREFACE.pdf
Saving 03 BRIEF INTRODUCTION.pdf to 03 BRIEF INTRODUCTION.pdf
Saving 05 TUITION FEE AND OTHER DUES.pdf to 05 TUITION FEE AND OTHER DUES.pdf
Saving 06 ADMISSION TO FIRST YEAR UG PROGRAM.pdf to 06 ADMISSION TO FIRST YEAR UG PROGRAM.pdf
Saving 07 ADMISSION IN SECOND YEAR OF UG PROGRAM (LATERAL ENTRY).pdf to 07 ADMISSION IN SECOND YEAR OF UG PROGRAM (LATERAL ENTRY).pdf
Saving 12 ADMISSION FOR FN-NRI CANDIDATES.pdf to 12 ADMISSION FOR FN-NRI CANDIDATES.pdf
Saving 13 INTERNATIONAL ENGINEERING PROGRAM.pdf to 13 INTERNATIONAL ENGINEERING PROGRAM.pdf
Saving 14 SCHOLARSHIP POLICY.pdf to 14 SCHOLARSHIP POLICY.pdf
Saving 15 FEATURES OF UG-PG PROGRAMS.pdf to 15 FEATURES OF UG-PG PROGRAMS.pdf
Saving 16 BRIEF INFORMATION ABOUT BIOMEDICAL ENGINEERING.pdf to 16 BRIEF INFORMATION ABOUT BIOMEDICAL ENGINEERING.pdf
Saving 18 BRIEF INFORMATION ABOUT SPECIALISATION COURSES IN BE (COMPUTER ENGINEERING).pdf to

5. Combine Web + Documents

In [ ]:
all_docs = web_docs + all_docs

6. Split into Chunks for Embedding

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(all_docs)

7. Create Embeddings + Chroma Vector Store

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(chunks, embeddings)

<ipython-input-8-a0d71d1e7f2b>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

8. LLM Integration + RetrievalQA

In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-pro")

qa_system = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

stream_handler = StreamingStdOutCallbackHandler()

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-pro",  # or "models/gemini-1.5-flash"
    google_api_key="your-API-key-here",
    streaming=True,
    callbacks=[stream_handler]
)


In [ ]:
from langchain.chains import RetrievalQA

qa_system = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

In [ ]:
from termcolor import colored
import time

print(colored("🤖 Welcome to the QA Bot! Type 'exit' to quit.\n", "cyan", attrs=["bold"]))

while True:
    query = input(colored("📝 Your Question: ", "light_blue"))

    if query.lower() in ["exit", "quit"]:
        print(colored("\n👋 Goodbye! Have a great day!", "magenta", attrs=["bold"]))
        break

    print(colored("💬 Thinking...", "yellow"))
    time.sleep(1)  # Simulate processing delay

    response = qa_system.invoke(query)

    print(colored("\n🔍 Final Answer:", "green", attrs=["bold"]))
    print(colored(response["result"], "white", attrs=["bold"]))
    print("\n" + "-"*60 + "\n")


🤖 Welcome to the QA Bot! Type 'exit' to quit.

📝 Your Question: email of Dean of Student Affairs
💬 Thinking...

🔍 Final Answer:
dosa@thapar.edu

------------------------------------------------------------

📝 Your Question: Seats in COE
💬 Thinking...

🔍 Final Answer:
Computer Engineering (COE) has 960 seats.

------------------------------------------------------------

📝 Your Question: How to calculate CGPA at Thapar
💬 Thinking...

🔍 Final Answer:
CGPA is calculated as the weighted average of all grades:

CGPA = (∑CiGi) / (∑Ci) 

Where:
* Ci is the number of credits assigned to the ith course
* Gi is the grade point equivalent to the letter grade obtained in the ith course.

'I' or 'X' grades are not included in CGPA calculations.  If a course is repeated, the new grade replaces the old one.  To convert CGPA to percentage, multiply by 10.

------------------------------------------------------------

📝 Your Question: Capacity of Vahni hall
💬 Thinking...

🔍 Final Answer:
Vahni Hall (fo